In [1]:
# importing the dependencies
from keras.models import load_model
from keras.preprocessing.image import img_to_array
import cv2
import numpy as np

In [2]:
model = load_model('models/Emotion_Detection_Model.h5')
classifier = cv2.CascadeClassifier("models/haarcascade_frontalface_default.xml")
labels = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']

In [3]:
def getBiggestFace(faces):
    area = []
    i = 0
    for x,y,w,h in faces:
        area.append((w*h, i));
    
    area = sorted(area, reverse = True)
    return area[0][1]

In [22]:
cap = cv2.VideoCapture(0)
while(True):
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray)
    if(len(faces) > 0):
        face_index = getBiggestFace(faces) #this will give us index of face that is closest to the camera
        face = faces[face_index]
        x,y,w,h = face  #unpacking
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48,48), interpolation = cv2.INTER_AREA) #INTER_AREA is an interpolation technique. If pixels are lost, then it will interpolate using the pixels in the surrounding 
        face = face.astype("float")/255.0  #first converting to float to avoid infomration loss
        face = img_to_array(face)
        face = np.expand_dims(face, axis = 0) # sort of like conversion to 2D array, so that we can use model prediction over it
        emotion = model.predict(face)[0] # predicting the emotion
        emotions = []
        for i in range(len(emotion)):
            emotions.append((labels[i],round(emotion[i]*100,2)))
        
        Emotions = []
        for emo in emotions:
            Emotions.append(emo[0] +": " + str(emo[1]) + "%")
        
        prediction = emotion.argmax() # gives index of emotion with highest percentage
        prediction = labels[prediction]
        
        
        cv2.rectangle(frame, (x-10, y-10), (x+w, y+h), (0,0,255), 3)
        cv2.putText(frame, prediction, (x-15, y-15), 2, 2, (0,0,255), 2, cv2.LINE_AA)
        
        displayHeight = 23
        
        # displaying the percentage of each emotion
        for Emo in Emotions:
            cv2.putText(frame, Emo, (x-15,y+h+displayHeight), 1, 2, (255,255,255), 2, cv2.LINE_AA)
            displayHeight += 30
    
    cv2.imshow("Emotion Detection", frame)
    if(cv2.waitKey(1) > 30):
        break
        

cap.release()
cv2.destroyAllWindows() 

1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 36ms/step
